1. Chargement des données

In [2]:
import pandas as pd

In [70]:
# Chargement des données
df = pd.read_csv('data.csv', encoding='ISO-8859-1')

# Aperçu
df.head()
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [71]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


2. Préparation des données

2.1.  Analyse exploratoire des données

In [72]:
#Structure et dimensions
df.shape
df.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

In [73]:
df.shape

(541909, 8)

In [74]:
#Statistiques descriptives
df.describe()
df['Country'].value_counts()

Country
United Kingdom          495478
Germany                   9495
France                    8557
EIRE                      8196
Spain                     2533
Netherlands               2371
Belgium                   2069
Switzerland               2002
Portugal                  1519
Australia                 1259
Norway                    1086
Italy                      803
Channel Islands            758
Finland                    695
Cyprus                     622
Sweden                     462
Unspecified                446
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
Israel                     297
USA                        291
Hong Kong                  288
Singapore                  229
Iceland                    182
Canada                     151
Greece                     146
Malta                      127
United Arab Emirates        68
European Community          61
RSA                         58


2.2. Nettoyage des données 

a. Valeurs manquantes

In [75]:
#Calcul du nombre de valeurs manquantes par colonne
missing_values = df.isnull().sum()

#Vérification s'il y a au moins une valeur manquante
if missing_values.any():
    # Colonne avec le plus de valeurs manquantes
    col_with_max_missing = missing_values.idxmax()
    num_missing = missing_values.max()
    
    print(f"✅ Oui, il y a des valeurs manquantes.")
    print(f"➡️ La colonne qui contient le plus de données manquantes est : '{col_with_max_missing}', avec {num_missing} valeurs manquantes.")
    print("\n📋 Détail des colonnes avec des valeurs manquantes :")
    print(missing_values[missing_values > 0])
else:
    print("✅ Non, il n'y a pas de valeurs manquantes dans le dataset.")

✅ Oui, il y a des valeurs manquantes.
➡️ La colonne qui contient le plus de données manquantes est : 'CustomerID', avec 135080 valeurs manquantes.

📋 Détail des colonnes avec des valeurs manquantes :
Description      1454
CustomerID     135080
dtype: int64


Traitement des valeurs manquantes

Lors de l’analyse des valeurs manquantes dans le dataset, nous avons constaté que seule la colonne CustomerID contenait des valeurs nulles (environ 135 000 sur 540 000 lignes). Cette colonne représente l’identifiant du client, essentiel dans le cadre d’une analyse de comportement client (segmentation, fidélité, etc.).

Nous avons choisi de supprimer les lignes contenant des valeurs manquantes dans CustomerID, car :

- Leur imputation n'est pas pertinente (il s'agit d’identifiants uniques, pas de valeurs numériques ou catégorielles classiques).

- Elles empêcheraient toute analyse centrée sur les clients.

- Elles ne représentent qu’environ 25 % du dataset total, donc leur suppression n’impacte pas significativement la représentativité globale.

En revanche, pour une analyse produit ou logistique, ces lignes auraient pu être conservées. Le choix dépend donc du cas d’usage spécifique.

In [9]:
# Supprimer les lignes où CustomerID est manquant (NaN)
#df = df.dropna(subset=['CustomerID'])

In [ ]:
#Nettoyage global
#df.dropna(subset=["InvoiceNo", "StockCode", "Description"], inplace=True)

b. Doublons

In [77]:
# Nombre de doublons exacts
nb_doublons = df.duplicated().sum()
print(f"Nombre de lignes dupliquées : {nb_doublons}")

#Suppression des doublons
df = df.drop_duplicates()
print("✅ Les doublons ont été supprimés.")


Nombre de lignes dupliquées : 5268
✅ Les doublons ont été supprimés.


In [78]:
df.shape

(535187, 8)

Traitement des doublons

Une vérification des doublons a révélé la présence de lignes strictement identiques. Celles-ci ont été supprimées afin d’éviter une sur-représentation artificielle de certaines transactions, ce qui pourrait fausser les statistiques de vente ou de comportement client. La suppression de ces doublons garantit l'intégrité des analyses futures.

c. Incohérences

- Valeurs négatives:

Quantity et UnitPrice doivent être positifs. Les valeurs négatives indiquent souvent des retours ou erreurs.

In [79]:
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]

- Transformation des types de colonnes

In [81]:
#Vérification et conversion du type de la colonne
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Vérification après conversion
print(df['InvoiceDate'].dtype)

# Convertir InvoiceNo en string 

df['InvoiceNo'] = df['InvoiceNo'].astype(str)


datetime64[ns]


Transformation des types de colonnes

- InvoiceDate a été convertie au format datetime, afin de permettre des analyses temporelles précises (extraction de la date, du mois, de l'heure, etc.).

- CustomerID était au format float64, ce qui n'est pas cohérent pour un identifiant. Après suppression des valeurs manquantes, il a été converti en chaîne de caractères (str) pour éviter toute confusion avec des valeurs numériques.

- InvoiceNo a été converti en str par sécurité, car certaines valeurs contiennent des lettres (ex. : les retours commencent par 'C').

2.3 Justification des transformations :

- Les valeurs manquantes dans CustomerID ont été supprimées, car cet identifiant est crucial pour les analyses client, et l’imputation n’est pas pertinente pour un champ de type ID.
- Les doublons exacts ont été supprimés afin de ne pas biaiser les statistiques de vente.
- Les transactions avec des Quantity ou UnitPrice négatifs ont été exclues, car elles représentent des retours ou des erreurs de saisie, et ne sont pas pertinentes dans une analyse des ventes.
- Le champ InvoiceDate a été converti au format datetime pour permettre une analyse temporelle ultérieure.


3. Création de nouvelles variables pertinentes

6 variables pertinentes avec leurs objectifs

In [82]:
# 1. Créer la variable TotalPrice (objectif : calcul du chiffre d'affaires et panier moyen)
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

In [83]:
# 2. Créer la variable IsCancelled (objectif : analyser les retours et qualité de service)
df['IsCancelled'] = df['InvoiceNo'].astype(str).str.startswith('C')

In [84]:
# 3. Extraire l’heure de la commande (objectif : analyser les pics d'activité/Compréhension des moments d’achat)
df['InvoiceHour'] = df['InvoiceDate'].dt.hour

In [85]:
#4 InvoiceTotalItems (Utilité : Évaluation du panier moyen par commande)
invoice_total_items = df.groupby('InvoiceNo')['Quantity'].sum().reset_index()
invoice_total_items.rename(columns={'Quantity': 'InvoiceTotalItems'}, inplace=True)
df = df.merge(invoice_total_items, on='InvoiceNo', how='left')

In [86]:
df.shape

(524878, 12)

In [87]:
# Maintenant qu'on a un dataset propre, on sépare :
df_products = df.copy()  # contient aussi les lignes sans client
df_clients = df.dropna(subset=['CustomerID']).copy()
df_clients['CustomerID'] = df_clients['CustomerID'].astype(int)



# 5. Variables spécifiques aux clients:
    #CustomerTotalSpent (Utilité : Identification des meilleurs clients)
customer_spending = df_clients.groupby('CustomerID')['TotalPrice'].sum().reset_index(name='CustomerTotalSpent')
df_clients = df_clients.merge(customer_spending, on='CustomerID', how='left')

    #CustomerNumOrders  (Utilité : Mesure de la fidélité ou fréquence d’achat)

customer_orders = df_clients[['CustomerID', 'InvoiceNo']].drop_duplicates()
customer_order_counts = customer_orders.groupby('CustomerID')['InvoiceNo'].count().reset_index(name='CustomerNumOrders')
df_clients = df_clients.merge(customer_order_counts, on='CustomerID', how='left')

 *fichier CSV du jeu de données final, nettoyé*
 on a separé le dataset en deux fichiers pour l'adapter aux analyses et à notre objectif de projet:
 - df_clients : Les valeurs manquantes dans CustomerID ont été supprimées, car cet identifiant est crucial pour les analyses client
 - dfproducts: contient aussi les lignes sans client

In [90]:
print(f'les colonnes de df_products:{df_products.columns}')
print(f'les dimensions de df_products :{df_products.shape}')

print(f'les colonnes de df_clients: {df_clients.columns}')
print(f'les dimensions de df_clients : {df_clients.shape}')



les colonnes de df_products:Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country', 'TotalPrice', 'IsCancelled',
       'InvoiceHour', 'InvoiceTotalItems'],
      dtype='object')
les dimensions de df_products :(524878, 12)
les colonnes de df_clients: Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country', 'TotalPrice', 'IsCancelled',
       'InvoiceHour', 'InvoiceTotalItems', 'CustomerTotalSpent',
       'CustomerNumOrders'],
      dtype='object')
les dimensions de df_clients : (392692, 14)


In [91]:
print("Taille de df avant toute transformation :", df.shape)

Taille de df avant toute transformation : (524878, 12)


4. Visualisation des tendances :